In [1]:
import sys
sys.path.append("../../")

from objects.node import Node
from objects.relationship import Relationship
from objects.data_model import DataModel
from objects.property import Property
from ingestion.generate_ingest import IngestionGenerator

In [2]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "password"
database = "neo4j"

In [3]:
person_name = Property(name="name", type="str", csv_mapping="name", is_unique=True)
person_age = Property(name="age", type="str", csv_mapping="age", is_unique=False)
address_street = Property(name="street", type="str", csv_mapping="street", is_unique=False)
address_city = Property(name="city", type="str", csv_mapping="city", is_unique=False)
pet_name = Property(name="name", type="str", csv_mapping="pet_name", is_unique=False)
pet_kind = Property(name="kind", type="str", csv_mapping="pet", is_unique=False)
toy_name = Property(name="name", type="str", csv_mapping="toy", is_unique=True)
toy_kind = Property(name="kind", type="str", csv_mapping="toy_type", is_unique=False)

In [4]:
nodes = [
            Node(
                label="Person",
                properties=[person_name, person_age],
                ),
            Node(
                label="Address", 
                properties=[address_street, address_city], 
            ),
            Node(
                label="Pet", 
                properties=[pet_name, pet_kind], 
            ),
            Node(
                label="Toy", 
                properties=[toy_name, toy_kind], 
            )
        ]

relationships = [
            Relationship(
                type="HAS_ADDRESS",
                properties=[],
                source="Person",
                target="Address",
            ),
            Relationship(
                type="KNOWS",
                properties=[],
                source="Person",
                target="Person",
            ),
            Relationship(
                type="HAS_PET",
                properties=[],
                source="Person",
                target="Pet",
            ),
            Relationship(
                type="PLAYS_WITH",
                properties=[],
                source="Pet",
                target="Toy",
            )
        ]

data_model = DataModel(nodes=nodes, relationships=relationships)

In [5]:
gen = IngestionGenerator(data_model=data_model.model_dump(), 
                         username=username, 
                         password=password, 
                         uri=uri, 
                         database=database, 
                         csv_file_path=None,
                         csv_name="test")

constraints:  {'person_name': 'CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;\n'}
constraints:  {'person_name': 'CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;\n'}
constraints:  {'person_name': 'CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;\n'}
constraints:  {'person_name': 'CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;\n', 'toy_name': 'CREATE CONSTRAINT toy_name IF NOT EXISTS FOR (n:Toy) REQUIRE n.name IS UNIQUE;\n'}
cypher map: 
 {'person': {'cypher': 'WITH $dict.rows AS rows\nUNWIND rows AS row\nMERGE (n:Person {name: row.name})\nSET n.age = row.age', 'cypher_loadcsv': "LOAD CSV WITH HEADERS FROM 'file:///file_name' as row\nCALL {\n\tWITH row\n\tMERGE (n:Person {name: row.name})\nSET n.age = row.age} IN TRANSACTIONS OF 10000 ROWS;\n", 'csv': '$BASE/test'}, 'address': {'cypher': 'WITH $dict.rows AS rows\nUNWIND rows AS row\nMERGE (n

TypeError: 'DataModel' object is not subscriptable

In [5]:
print(gen.generate_pyingest_yaml(write_file=False))

server_uri: bolt://localhost:7687
admin_user: neo4j
admin_pass: password
basepath: file:./

pre_ingest:
  - CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;
  - CREATE CONSTRAINT address_name IF NOT EXISTS FOR (n:Address) REQUIRE n.name IS UNIQUE;
  - CREATE CONSTRAINT pet_name IF NOT EXISTS FOR (n:Pet) REQUIRE n.name IS UNIQUE;
  - CREATE CONSTRAINT toy_name IF NOT EXISTS FOR (n:Toy) REQUIRE n.name IS UNIQUE;
files:
- chunk_size: 100
  cql: |-
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Person {name: row.name})
    SET n.age = row.age
  url: $BASE/data/None
- chunk_size: 100
  cql: |-
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Address {name: row.name})
    SET n.street = row.street, n.city = row.city
  url: $BASE/data/None
- chunk_size: 100
  cql: |-
    WITH $dict.rows AS rows
    UNWIND rows AS row
    MERGE (n:Pet {name: row.name})
    SET n.kind = row.kind, n.name = row.name
  url: $BASE/data/None
- chunk

In [6]:
print(gen.generate_constraints_cypher())

CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT address_name IF NOT EXISTS FOR (n:Address) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT pet_name IF NOT EXISTS FOR (n:Pet) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT toy_name IF NOT EXISTS FOR (n:Toy) REQUIRE n.name IS UNIQUE;



In [7]:
print(gen.generate_load_csv())

CREATE CONSTRAINT person_name IF NOT EXISTS FOR (n:Person) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT address_name IF NOT EXISTS FOR (n:Address) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT pet_name IF NOT EXISTS FOR (n:Pet) REQUIRE n.name IS UNIQUE;
CREATE CONSTRAINT toy_name IF NOT EXISTS FOR (n:Toy) REQUIRE n.name IS UNIQUE;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Person {name: row.name})
SET n.age = row.age} IN TRANSACTIONS OF 10000 ROWS;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Address {name: row.name})
SET n.street = row.street, n.city = row.city} IN TRANSACTIONS OF 10000 ROWS;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Pet {name: row.name})
SET n.kind = row.kind, n.name = row.name} IN TRANSACTIONS OF 10000 ROWS;
LOAD CSV WITH HEADERS FROM 'file:///file_name' as row
CALL {
	WITH row
	MERGE (n:Toy {name: row.name})
SET n.kind = row.kind} IN TRANSACTIONS OF 10000 R